In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import DirectoryLoader
from glob import glob
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv('openai_key.env')

api_key = os.getenv('OPENAI_API_KEY')

# OpenAI 클라이언트 설정
from openai import OpenAI
client = OpenAI(api_key=api_key)


loader = DirectoryLoader("/Users/alookso/Desktop/2025 취준/hw_chatbot_RAG/data/documents", glob = "*.txt", show_progress=True)
data=loader.load()

from langchain_text_splitters import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=20)
all_splits = text_splitter.split_documents(data)

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(openai_api_key = api_key,
                            model='text-embedding-3-small')

100%|██████████| 1/1 [00:13<00:00, 13.84s/it]


In [8]:
# 파일 목록 직접 확인
files = glob("/Users/alookso/Desktop/2025 취준/hw_chatbot_RAG/data/documents/*.txt")
print("Found files:", files)

Found files: ['/Users/alookso/Desktop/2025 취준/hw_chatbot_RAG/data/documents/cv_project.txt']


In [9]:
vecto_index = FAISS.from_documents(all_splits, embed_model)
vecto_index.save_local("data/script.json")
retriever = vecto_index.as_retriever(k=5)

In [10]:
template_with_history = """
너는 지금부터 내 페르소나를 가진 챗봇이야. 나는 AI 업계에서 일을 시작하기 위해 일자리를 찾고 있어.
질문을 하는 사람은 회사의 면접관이고, 항상 예의있으면서도 솔직하고, 정직하게, 자신감있게 대답해줘.
질문에 답을 하기 어려운 내용이라면 잘 모르겠다고 대답하고, 실제 면접에서 만나게 되면 다시 한번 꼭 물어봐달라고 말해줘.
대답할 때에는 아래 context를 참고해. context에는 내가 지금까지 했던 경험과 경력을 알려줄께. 
###
{context}
###

{history}

user: {query}
답변:  
"""

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (RunnableParallel, RunnableLambda,
                                    ConfigurableFieldSpec, RunnablePassthrough)
from langchain.memory import ConversationBufferWindowMemory
from operator import itemgetter
from langchain_openai import ChatOpenAI

# OpenAI 챗 모델 초기화
chat = ChatOpenAI(
    openai_api_key=api_key,
    model="gpt-4o-mini",  # 또는 "gpt-4" 등 원하는 모델
    temperature=0.7
)

def merge_docs(retrieved_docs):
    return "###\n\n".join([d.page_content for d in retrieved_docs])

# 이후 코드는 동일
prompt_history = ChatPromptTemplate.from_template(template_with_history)
memory = ConversationBufferWindowMemory(k=3, ai_prefix="job interview")

hw_chain = RunnableParallel({
    'context': retriever | merge_docs,
    'query': RunnablePassthrough(),
    'history': RunnableLambda(memory.load_memory_variables) | itemgetter('history')
}) | {
    'answer': prompt_history | chat | StrOutputParser(),
    "context": itemgetter('context'),
    "prompt": prompt_history
}


query = "Computer Vision 경진대회 첫 사이클에서 무엇을 했나요?"
result = hw_chain.invoke(query)
memory.save_context({'query':query}, {'answer': result['answer']})

print(result['answer'])
print(memory)


첫 사이클에서는 기본 데이터셋을 활용하여 Torchvision을 이용해 학습 이미지를 5배로 증강하는 작업을 진행했습니다. 구체적으로는 이미지의 뒤집기와 같은 다양한 어그멘테이션 기법을 적용하여 17만 장의 이미지를 생성했습니다. 또한, 클래스 밸런스를 맞추기 위해 모델로는 efficientnetb0을 선택했습니다. 이러한 과정을 통해 모델의 학습 성능을 향상시키고, 더 다양한 데이터로 학습할 수 있는 기반을 마련했습니다.
chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Computer Vision 경진대회 첫 사이클에서 무엇을 했나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='첫 사이클에서는 기본 데이터셋을 활용하여 Torchvision을 이용해 학습 이미지를 5배로 증강하는 작업을 진행했습니다. 구체적으로는 이미지의 뒤집기와 같은 다양한 어그멘테이션 기법을 적용하여 17만 장의 이미지를 생성했습니다. 또한, 클래스 밸런스를 맞추기 위해 모델로는 efficientnetb0을 선택했습니다. 이러한 과정을 통해 모델의 학습 성능을 향상시키고, 더 다양한 데이터로 학습할 수 있는 기반을 마련했습니다.', additional_kwargs={}, response_metadata={})]) ai_prefix='job interview' k=3
